# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [ ]:
from copy import copy

class Matrix:

    # Constructor handling multiple initialization methods
    def __init__(self, *args):
        if len(args) == 1:
            data = args[0]
            if isinstance(data, list):
                if all(isinstance(i, list) for i in data):
                    self.__n = len(data)
                    self.__m = len(data[0])
                    self.M = data
                else:
                    # Convert single list to a list of lists (row vector)
                    self.__n = 1
                    self.__m = len(data)
                    self.M = [data]
            else:
                raise ValueError("Invalid initialization parameters")
        elif len(args) == 2:
            n, m = args
            if isinstance(n, int) and isinstance(m, int):
                self.__n = n
                self.__m = m
                self.M = [[0] * self.__m for _ in range(self.__n)]
            else:
                raise ValueError("Invalid initialization parameters")
        elif len(args) == 3:
            n, m, value = args
            if isinstance(n, int) and isinstance(m, int):
                self.__n = n
                self.__m = m
                self.M = [[value] * self.__m for _ in range(self.__n)]
            else:
                raise ValueError("Invalid initialization parameters")
        else:
            raise ValueError("Invalid number of arguments")


    # Magic methods for indexing, addition, subtraction, multiplication, and equality
    def __getitem__(self, index):
        if isinstance(index, tuple):
            i, j = index
            return self.M[i][j]
        elif isinstance(index, slice):
            return Matrix(self.M[index])
        elif isinstance(index, int):
            return self.M[index]
        else:
            raise TypeError("Invalid index type")
    
    def __setitem__(self, index, value):
        if isinstance(index, tuple):
            i, j = index
            self.M[i][j] = value
        elif isinstance(index, int):
            self.M[index] = value
        else:
            raise TypeError("Invalid index type")

    # Magic method for addition, using add method
    def __add__(self, other):
        if(isinstance(other, Matrix)):
            result = Matrix(self.M)
            result.add(other)
            return result
        
    # Magic method for subtraction, using subtract method
    def __sub__(self, other):
        if(isinstance(other, Matrix)):
            result = Matrix(self.M)
            result.sub(other)
            return result
    
    # Magic method for multiplication
    def __mul__(self, other):
        if(isinstance(other, Matrix)):
            result = Matrix(self.M)
            result.mat_mult(other)
            return result
        elif(isinstance(other, int) or isinstance(other, float)):
            result = Matrix(self.M)
            result.scalar_mul(other)
            return result
        
    # Reversed multiplication for scalar multiplication
    def __rmul__(self, other):
        return self.__mul__(other)
    
    # Magic method for equality, ensures both objects are of type Matrix, and uses equals method
    def __eq__(self, other):
        if(isinstance(other, Matrix)):
            return self.equals(other)
        raise TypeError("Both objects must be of type Matrix")
        
    # Magic method for copying if a matrix is not been created yet, but wants to just copy another matrix
    def __copy__(self):
            return type(self)(self.M)
    
    # Copy method for copying the contents of one matrix to another
    def copy(self, other):
        if isinstance(other, Matrix):
            if self.check_shape(other):
                self.M = other.M
        else:
            raise TypeError("Both objects must be of type Matrix")
        
    # Non-magic methods for matrix operations

    # Methods for getting matrix properties
    def shape(self):
        return (self.__n, self.__m)
    
    # Methods for geting transpose, row, column, and block of the matrix

    # Returns the transpose of the matrix as a new Matrix object
    def transpose(self):
        transposed = [[self.M[j][i] for j in range(self.__n)] for i in range(self.__m)]
        return Matrix(transposed)

    # Returns the i-th row of the matrix as a new Matrix object
    def row(self, i):
        return Matrix(self.M[i-1])
    
    # Returns the j-th column of the matrix as a new Matrix object
    def column(self, j):
        col = [[self.M[i][j-1]] for i in range(self.__n)]
        return Matrix(col)
    
    # Returns the matrix as a list of lists
    def to_list(self):
        return self.M

    # Returns a block of the matrix as a new Matrix object
    def block(self, n_0, n_1, m_0, m_1):
        block = [row[m_0:m_1] for row in self.M[n_0:n_1]]
        return Matrix(block)
    
    # Methods for creating matrices of constants, zeroes, ones, and identity matrix

    # Creates a matrix of constants
    def constant(self, n, m, c):
        return Matrix(n, m, c)
    
    # Creates a matrix of zeroes, handled by the constructor
    def zeroes(self, n, m):
        return Matrix(n, m)
    
    # Creates a matrix of ones, using the constant method
    def ones(self, n, m):
        return Matrix(n, m, 1)
    
    # Creates an identity matrix of the given size
    def eye(self, n):
        eye = Matrix(n, n)
        for i in range(n):
            eye[i, i] = 1
        return eye

    # Methods for scalar multiplication, addition, subtraction, matrix multiplication, and element-wise multiplication

    # Scalars the matrix by a constant
    def scalar_mul(self, c):
        self.M = [[c * self.M[i][j] for j in range(self.__m)] for i in range(self.__n)]
    
    # Checks if the shape of two matrices are the same
    def check_shape(self, other):
        if self.shape() == other.shape():
            return True
        raise ValueError("Matrices must have the same shape")

    # Adds another matrix to the current matrix, makes sure the shapes are the same
    def add(self, other):
        if(self.check_shape(other)):
            self.M = [[self.M[i][j] + other[i, j] for j in range(self.__m)] for i in range(self.__n)]
    
    # Subtracts another matrix from the current matrix, makes sure the shapes are the same
    def sub(self, other):
        if(self.check_shape(other)):
            self.M = [[self.M[i][j] - other[i, j] for j in range(self.__m)] for i in range(self.__n)]
    
    # Multiplies the current matrix by another matrix, checks if the shapes are the same, and if not, checks if the inner dimensions are the same
    def mat_mult(self, other):
        try: 
            self.check_shape(other)
        except ValueError as e:
            _, m1m = self.shape()
            m2n, _ = other.shape()
            if(m1m != m2n):
                raise e
        finally:
            self.M = [[sum([self.M[i][k] * other[k, j] for k in range(self.__m)]) for j in range(other.shape()[1])] for i in range(self.__n)]

    # Element-wise multiplication of two matrices, makes 
    def element_mult(self, other):
        if(self.check_shape(other)):
            self.M =[[self.M[i][j] * other[i, j] for j in range(self.__m)] for i in range(self.__n)]

    # Checks if two matrices are equal, makes sure the shapes are the same
    def equals(self, other):
        if(self.check_shape(other)):
            return all([self.M[i][j] == other[i, j] for i in range(self.__n) for j in range(self.__m)])
    
    # String representation of the matrix
    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.M])

# Test Matricies
M = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
N = copy(M)

print("Demonstration of Matrix class methods, first we will show the original matrices:\n")

print("Original Matrix M:")
print(M)

print("Original Matrix N:")
print(N)

print("Operations will be performed on Matrix M, and the results will be shown below, first the non-magic methods:")

# Demonstrated all methods are working by order of instruction in assignment:

# Shape
Mn, Mm = M.shape()
print(f"\nShape of M: {Mn}x{Mm}")

# Transpose
T = M.transpose()
print("\nTranspose of M:")
print(T)

# Row
print("\nRow 1 of M:")
print(M.row(1))

# Column
print("\nColumn 1 of M:")
print(M.column(1))

# To list
print("\nM as list:")
print(M.to_list())

# Block
print("\nBlock of M:")
print(M.block(0, 2, 1, 3))

# Constant
print("\nMatrix of constants:")
print(M.constant(3, 3, 5))

# Zeroes
print("\nMatrix of zeroes:")
print(M.zeroes(3, 3))

# Ones
print("\nMatrix of ones:")
print(M.ones(3, 3))

# Eye
print("\nIdentity matrix:")
print(M.eye(3))

# The following methods are tested by just the methods and not magic methods:
# Scalar multiplication
test = copy(M)
test.scalar_mul(2)
print("\nM * 2:")
print(M)

# Add
test = copy(M)
test.add(N)
print("\nM + N:")
print("Matrix M: + Matrix N = Matrix M + N")
print(f"{test}")

# Subtract
test = copy(M)
test.sub(N)
print("\nM - N:")
print("Matrix M - Matrix N = Matrix M - N")
print(f"{test}")

# Matrix multiplication
test = copy(M)
test = M * N
print("\nMatrix multiplication of M and N:")
print(f"{test}")

# Element-wise multiplication
test = copy(M)
test.element_mult(N)
print("\nElement-wise multiplication of M and N:")
print(f"{test}")

# Equality
test = copy(M)
print("\nMatrix M == Matrix N")
print(f"{test.equals(N)}")

print("\nNow we will demonstrate the magic methods:")

# Multiplication
test = copy(M)
test = M * N
print("\nMatrix multiplication of M and N:")
print(f"{test}")

# Multiplication with scalar
test = copy(M)
test = M * 2
print("\nMatrix M * 2:")
print(f"{test}")

# Multiplication with scalar (reversed)
test = copy(M)
test = 2 * M
print("\nMatrix 2 * M:")
print(f"{test}")

# Addition
test = copy(M)
test = M + N
print("\nMatrix M + N:")
print(f"{test}")

# Subtraction
test = copy(M)
test = M - N
print("\nMatrix M - N:")
print(f"{test}")

# Equality
test = copy(M)
print("\nMatrix M == Matrix N")
print(f"{test == N}")

# Test copy method
N = Matrix([[9,8,7],[6,5,4],[3,2,1]])

print("\nOriginal Matrix M:")
print(M)
print("Original Matrix N:")
print(N)

M.copy(N)

print("\nMatrix M after copying N:")
print(M)

print("\nAll methods have been demonstrated successfully.")

# Demonstrate matrix properties
print("\nMatrix properties:")

print("(AB)C == A(BC):\n")
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])
C = Matrix([[9, 10], [11, 12]])

ABC1 = (A * B) * C
ABC2 = A * (B * C)
print(f"(AB)C: \n{(A * B) * C}")
print(f"\nA(BC): \n{A * (B * C)}")

print(f"(AB)C == A(BC): {ABC1 == ABC2}")

print("\nA(B+C) == AB + AC:\n")

AB_C1 = A * (B + C)
AB_C2 = (A * B) + (A * C)

print(f"A(B+C): \n{A * (B + C)}")
print(f"\nAB + AC: \n{(A * B) + (A * C)}")

print(f"\nA(B+C) == AB + AC: {AB_C1 == AB_C2}")

print("\nAB != BA:\n")

AB = A * B
BA = B * A

print(f"AB: \n{A * B}")
print(f"\nBA: \n{B * A}")

print(f"\nAB != BA: {AB != BA}")

print("\nAI = A:\n")
AI = A * A.eye(2)

print(f"AI: \n{AI}")
print(f"\nA: \n{A}")
print(f"\nAI == A: {AI == A}")


print("All general matrix properties have been demonstrated successfully.")


Demonstration of Matrix class methods, first we will show the original matrices:

Original Matrix M:
1 2 3
4 5 6
7 8 9
Original Matrix N:
1 2 3
4 5 6
7 8 9
Operations will be performed on Matrix M, and the results will be shown below, first the non-magic methods:

Shape of M: 3x3

Transpose of M:
1 4 7
2 5 8
3 6 9

Row 1 of M:
1 2 3

Column 1 of M:
1
4
7

M as list:
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

Block of M:
2 3
5 6

Matrix of constants:
5 5 5
5 5 5
5 5 5

Matrix of zeroes:
0 0 0
0 0 0
0 0 0

Matrix of ones:
1 1 1
1 1 1
1 1 1

Identity matrix:
1 0 0
0 1 0
0 0 1

M * 2:
1 2 3
4 5 6
7 8 9

M + N:
Matrix M: + Matrix N = Matrix M + N
2 4 6
8 10 12
14 16 18

M - N:
Matrix M - Matrix N = Matrix M - N
0 0 0
0 0 0
0 0 0

Matrix multiplication of M and N:
30 36 42
66 81 96
102 126 150

Element-wise multiplication of M and N:
1 4 9
16 25 36
49 64 81

Matrix M == Matrix N
True

Now we will demonstrate the magic methods:

Matrix multiplication of M and N:
30 36 42
66 81 96
102 126 150

Matrix 